In [1]:
%load_ext autoreload
%autoreload 2

import sys
import cv2
import numpy as np
sys.path.insert(1, '../../Src/')
from utils import vision
from utils import camera
import time
import traceback

COLOR_BOUNDINGBOX = (255, 0, 0)
THICKNESS_BOUNDINGBOX = 2


In [2]:
from models.faceboxes.faceboxes_onnx import FaceBoxes_onnx

In [3]:
face_boxes = FaceBoxes_onnx.FaceBoxes_ONNX(timer_flag=False)

In [4]:
cap = camera.ParallelCamera(0).start()

start webcam


In [5]:


my_fps = vision.FPS(100)

my_fps_model = vision.FPS(100)


while True:
  my_fps.start()

  # Read image
  success, image = cap.read()  
  # Flip the image horizontally for a selfie-view display.    
  image = cv2.flip(image.copy(), 1)

  image = cv2.resize(image,(300,300))
  #time.sleep(0.5)

  if not success:
    print("Ignoring empty camera frame.")
    continue

  my_fps_model.start()
  
  # Model Inference
  image.flags.writeable = False
  #predictions = model.predict(image)
  boxes, scores= face_boxes.predict(image)
  image.flags.writeable = True  
  
  # Draw bounding boxes
  image = vision.draw_boundingboxes(image,boxes,COLOR_BOUNDINGBOX,THICKNESS_BOUNDINGBOX,if_wh=False)
  
  # Calculate FPS
  my_fps_model.update()
  my_fps.update()
  # Show image
  image = vision.put_text(image,"FPS      : "+str(np.round(my_fps.fps(),3)),pos =(20,20))
  image = vision.put_text(image,"FPS model: "+str(np.round(my_fps_model.fps(),3)),pos =(20,50))
  cv2.imshow('Face Detection Project',image)
 
  # Close window with ESC
  if cv2.waitKey(5) & 0xFF == 27:
    break

# DestroyWindows
cap.release()
cv2.destroyAllWindows()
